In [1]:
import cv2
import pandas as pd
import numpy as np
import json

In [2]:
with open('bboxes.json', 'r') as j:
     contents = json.loads(j.read())

In [3]:
def convert_to_yolo_box_params(box_coordinates, im_w, im_h):

        ans = []

        ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)  # x_center
        ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
      
        ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)  # width
        ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height
        return ans

In [11]:
def show_frames_w_bboxes(trim_name, df):
    vidcap = cv2.VideoCapture(f'popik/Локотех_{trim_name}.mp4')
    success, image = vidcap.read()
    count = 1
    
    while success:
        objects = df[df.frame == count]
        for _, obj in objects.iterrows():
            print(obj)
            name = obj.label
        
            if name == 'bot':
                color = (0, 230, 255)
            elif name == 'mob':
                color = (188, 0, 255)
            else:
                color = (255, 255, 255)
                
            t = int(obj.top)
            h = int(obj.height)
            l = int(obj.left)
            w = int(obj.width)
            
            cv2.rectangle(image, (l + w, t), (l, t + h), color, 2)
            
            cv2.imshow('lol', image)
            cv2.waitKey(0)
            cv2.destroyWindow('lol')
            success, image = vidcap.read()
        count += 1

In [50]:
def show_frames_w_bboxes(trim_name):
    frames = contents[int(trim_name[-1]) - 1]['projects.clgwbo68i04k1071t3d8i4qo1.labels'][0]['annotations']['frames']
    vidcap = cv2.VideoCapture(f'popik/Локотех_{trim_name}.mp4')
    success,image = vidcap.read()
    count = 1
    while success:
        if frames.get(str(count)) is not None:
            objects = frames.get(str(count)).get('objects').keys()
            for obj in objects:
                bb = frames.get(str(count)).get('objects').get(obj).get('bounding_box')
                name = frames.get(str(count)).get('objects').get(obj).get('name')

                if name == 'bot':
                    color = (0, 230, 255)
                elif name == 'mob':
                    color = (188, 0, 255)
                else:
                    color = (255,255,255)

                t = int(bb['top'])
                h = int(bb['height'])
                l = int(bb['left'])
                w = int(bb['width'])                
                cv2.rectangle(image, (l + w, t), (l, t + h), color, 2)
                
                # print(l, t)
                # print(l, t-h)
                # print(l+w, t)
                # print(l+w, t-h)
                
        else:
            bb = None

        cv2.imshow("hh", image)     # save frame as JPEG file     
        cv2.waitKey(0) 
        cv2.destroyWindow("hh") #make sure window closes cleanly
        success,image = vidcap.read()
        count += 1

In [ ]:
show_frames_w_bboxes('Trim3', test)

frame                                1
object_id    clgwczv1p001a356jsdzjjxll
label                              bot
top                              528.0
left                             331.0
height                           191.0
width                            133.0
Name: 0, dtype: object
frame                                1
object_id    clgwd3nhy001k356j4eixwwax
label                              bot
top                              168.0
left                             573.0
height                            67.0
width                             23.0
Name: 1, dtype: object
frame                                1
object_id    clgwde66w001o356jka0sdgs6
label                              mob
top                              191.0
left                             600.0
height                            92.0
width                             33.0
Name: 2, dtype: object
frame                                1
object_id    clgwde9rc001s356joqdedjv9
label                             

In [141]:
# one = contents[2]['projects.clgwbo68i04k1071t3d8i4qo1.labels'][0]['annotations']['frames'].get('1')['objects']['clgwczv1p001a356jsdzjjxll'].get('bounding_box')
# two = contents[2]['projects.clgwbo68i04k1071t3d8i4qo1.labels'][0]['annotations']['frames'].get('6')['objects']['clgwczv1p001a356jsdzjjxll'].get('bounding_box')

# # Define the start and end bounding boxes
# start_bbox = np.array([one['left'], one['top'], one['width'], one['height']])
# end_bbox = np.array([two['left'], two['top'], two['width'], two['height']])

# # Define the number of intermediate frames
# num_frames = 4

# # Calculate the time interval between the start and end frames
# time_interval = 1.0 / (num_frames + 1)

# # Generate intermediate frames
# intermediate_frames = []
# for i in range(num_frames):
#     # Calculate the percentage of time that has elapsed
#     elapsed_time = (i + 1) * time_interval

#     # Interpolate the bbox position using linear interpolation
#     bbox_position = start_bbox[:2] + (end_bbox[:2] - start_bbox[:2]) * elapsed_time

#     # Interpolate the bbox size using linear interpolation
#     bbox_size = start_bbox[2:] + (end_bbox[2:] - start_bbox[2:]) * elapsed_time

#     # Combine the interpolated position and size to create the interpolated bbox
#     interpolated_bbox = np.concatenate([bbox_position, bbox_size])

#     # Add the interpolated bbox to the list of intermediate frames
#     intermediate_frames.append(interpolated_bbox)

In [5]:
frames = contents[2]['projects.clgwbo68i04k1071t3d8i4qo1.labels'][0]['annotations']['frames']  # delete

for i in range(int([*frames][0]) + 1, int([*frames][1])):
    
    objects_start = frames.get([*frames][0]).get('objects').keys()
    objects_end = frames.get([*frames][1]).get('objects').keys()

    start_set = set()
    end_set = set()

    for obj_s, obj_e in zip(objects_start, objects_end):
        start_set.add(obj_s)
        end_set.add(obj_e)

    objects_to_keep = start_set.intersection(end_set)
    
    for obj_id in objects_to_keep:
        
        one = frames.get([*frames][0])['objects'][obj_id].get('bounding_box')
        two = frames.get([*frames][1])['objects'][obj_id].get('bounding_box')

        # Define the start and end bounding boxes
        start_bbox = np.array([one['left'], one['top'], one['width'], one['height']])
        end_bbox = np.array([two['left'], two['top'], two['width'], two['height']])

        # Define the number of intermediate frames
        num_frames = 4

        # Calculate the time interval between the start and end frames
        time_interval = 1.0 / (num_frames + 1)

        # Generate intermediate frames
        intermediate_frames = []
        for j in range(num_frames):
            # Calculate the percentage of time that has elapsed
            elapsed_time = (j + 1) * time_interval

            # Interpolate the bbox position using linear interpolation
            bbox_position = start_bbox[:2] + (end_bbox[:2] - start_bbox[:2]) * elapsed_time

            # Interpolate the bbox size using linear interpolation
            bbox_size = start_bbox[2:] + (end_bbox[2:] - start_bbox[2:]) * elapsed_time

            # Combine the interpolated position and size to create the interpolated bbox
            interpolated_bbox = np.concatenate([bbox_position, bbox_size])

            # Add the interpolated bbox to the list of intermediate frames
            intermediate_frames.append(interpolated_bbox)
        
        for p in intermediate_frames:
            
         #   print('frame', i, 'coords', p, 'object id', obj_id)
            obj_name = frames.get([*frames][0]).get('objects').get(obj_id).get('name')
            
            frames[str(i)]['objects'] = {obj_id: {'name': obj_name,
                                                  'bounding_box': {}}}
          #  frames[str(i)]['objects'][obj_id]['name'] = obj_name
            frames[str(i)]['objects'][obj_id]['bounding_box']['top'] = p[1]
            frames[str(i)]['objects'][obj_id]['bounding_box']['left'] = p[0]
            frames[str(i)]['objects'][obj_id]['bounding_box']['height'] = p[3]
            frames[str(i)]['objects'][obj_id]['bounding_box']['width'] = p[2]
            
          #  print(frames[str(i)]['objects'][obj_id]['bounding_box'])
            
            # frames.update({f'{i}': 
            #                   {'objects': 
            #                       {f'{obj_id}': 
            #                           {'name': f'{obj_name}',
            #                            'bounding_box': {'top': p[1],
            #                                             'left': p[0],
            #                                             'height': p[3],
            #                                             'width': p[2]}}}}})
            
          #  print(i, 'saved obj id', obj_id, 'name', obj_name)

KeyError: '2'

In [6]:
its = list(contents[2]['projects.clgwbo68i04k1071t3d8i4qo1.labels'][0]['annotations']['frames'].items())

In [7]:
matrix = []
for frame in its:
    frame_num = frame[0]
    objects = frame[1]['objects']
    obj_id = objects.keys()

    for aidi in obj_id:
        row = []
        bbox = [k[1] for k in [*objects[aidi]['bounding_box'].items()]]
        row.append([frame_num, aidi, objects[aidi]['name'], *bbox])
        matrix.extend(row)

In [8]:
df = pd.DataFrame(matrix, columns=['frame', 'object_id', 'label', 'top', 'left', 'height', 'width'])
df.frame = df.frame.astype(int)

In [9]:
test = df.copy()

for first, second in zip(df.frame.unique().tolist(), df.frame.unique().tolist()[1:]):
    tmp = df[(df.frame==first) | (df.frame==second)]
    desired_range = range(first+1, second)
    for _, row in tmp[tmp.frame==second].iterrows():
        obj_id = row.object_id
        if obj_id in tmp[tmp.frame==second].object_id.tolist():
            obj_name = row.label
            obj_df = pd.DataFrame({'frame': desired_range,
                                    'object_id': [obj_id] * len(desired_range),
                                    'label': [obj_name] * len(desired_range)})
            
            obj_df['top'] = None
            obj_df['left'] = None
            obj_df['height'] = None
            obj_df['width'] = None
            inter = test[(test['frame']==first) | (test['frame']==second)]
            inter = pd.concat([inter, obj_df], axis=0)
            inter = inter.interpolate()
            test = pd.concat([test, inter], axis=0)
            
test = test.drop_duplicates().sort_values('frame').reset_index(drop=True)

In [10]:
test

,frame,object_id,label,top,left,height,width
0,1,clgwczv1p001a356jsdzjjxll,bot,528.0,331.0,191.0,133.0
1,1,clgwd3nhy001k356j4eixwwax,bot,168.0,573.0,67.0,23.0
2,1,clgwde66w001o356jka0sdgs6,mob,191.0,600.0,92.0,33.0
3,1,clgwde9rc001s356joqdedjv9,mob,183.0,640.0,60.0,22.0
4,2,clgwd3nhy001k356j4eixwwax,bot,169.0,573.0,67.0,25.0
...,...,...,...,...,...,...,...
5653,2637,clgwde9rc001s356joqdedjv9,mob,171.0,632.0,61.0,29.0
5654,2638,clgwde9rc001s356joqdedjv9,mob,171.0,632.0,61.0,29.0
5655,2639,clgwde9rc001s356joqdedjv9,mob,171.0,632.0,61.0,29.0
5656,2640,clgwde9rc001s356joqdedjv9,mob,171.0,632.0,61.0,29.0
